In [94]:
import h5py
import librosa
import numpy      as np
import pandas     as pd
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

In [95]:
dir = 'C:/COUGHVID_public_dataset/public_dataset/'

# read in metadata and extract unique identifiers
data = pd.read_csv(dir+'metadata_compiled.csv')
#uuids = data.get('uuid').to_list()

# get only data that has a COVID status label. Loading all the files takes too long
labelled_data = data[ np.isin(data['status'],['healthy','symptomatic','COVID-19']) ]

uuids = labelled_data.get('uuid').to_list()

print(str(len(uuids)) + " matching records found.")

16224 matching records found.


In [ ]:
''' WARNING: THIS CELL TAKES AT LEAST 12 HOURS TO RUN '''
import pydub

# open sound files
extensions = ['.webm','.ogg']

sounds = []
#sample_rates = []

for i,uuid in enumerate(uuids):
    print(str(i+1) + " of " + str(len(uuids)) + " processed. This may take awhile...")
    
    sound = None
    #sample_rate = 0
    for ext in extensions:
        try:
            #sound, sample_rate = librosa.load(dir+uuid+ext,sr=None)
            # much faster than librosa
            sound = pydub.AudioSegment.from_file(dir+uuid+ext).get_array_of_samples()
            break
        except FileNotFoundError as e:
            print("File " + dir+uuid+ext + " not found. Trying next extension...")
            
    sounds       += [sound]
    #sample_rates += [sample_rate]
            

1 of 16224 processed. This may take awhile...
2 of 16224 processed. This may take awhile...
3 of 16224 processed. This may take awhile...
4 of 16224 processed. This may take awhile...
5 of 16224 processed. This may take awhile...
6 of 16224 processed. This may take awhile...
7 of 16224 processed. This may take awhile...
8 of 16224 processed. This may take awhile...
9 of 16224 processed. This may take awhile...
10 of 16224 processed. This may take awhile...
11 of 16224 processed. This may take awhile...
12 of 16224 processed. This may take awhile...
13 of 16224 processed. This may take awhile...
14 of 16224 processed. This may take awhile...
15 of 16224 processed. This may take awhile...
16 of 16224 processed. This may take awhile...
17 of 16224 processed. This may take awhile...
18 of 16224 processed. This may take awhile...
File C:/COUGHVID_public_dataset/public_dataset/005887c9-4bb1-4f13-86b2-1c7b3cee0881.webm not found. Trying next extension...
19 of 16224 processed. This may take a

156 of 16224 processed. This may take awhile...
157 of 16224 processed. This may take awhile...
158 of 16224 processed. This may take awhile...
159 of 16224 processed. This may take awhile...
160 of 16224 processed. This may take awhile...
161 of 16224 processed. This may take awhile...
162 of 16224 processed. This may take awhile...
163 of 16224 processed. This may take awhile...
File C:/COUGHVID_public_dataset/public_dataset/029685b5-d06b-44ea-9ad9-6f6134438c67.webm not found. Trying next extension...
164 of 16224 processed. This may take awhile...
File C:/COUGHVID_public_dataset/public_dataset/02986b04-14ae-4331-a783-9245f4cc7786.webm not found. Trying next extension...
165 of 16224 processed. This may take awhile...
166 of 16224 processed. This may take awhile...
167 of 16224 processed. This may take awhile...
168 of 16224 processed. This may take awhile...
169 of 16224 processed. This may take awhile...
170 of 16224 processed. This may take awhile...
171 of 16224 processed. This m

305 of 16224 processed. This may take awhile...
306 of 16224 processed. This may take awhile...
307 of 16224 processed. This may take awhile...
308 of 16224 processed. This may take awhile...
309 of 16224 processed. This may take awhile...
310 of 16224 processed. This may take awhile...
311 of 16224 processed. This may take awhile...
312 of 16224 processed. This may take awhile...
313 of 16224 processed. This may take awhile...
314 of 16224 processed. This may take awhile...
315 of 16224 processed. This may take awhile...
316 of 16224 processed. This may take awhile...
317 of 16224 processed. This may take awhile...
318 of 16224 processed. This may take awhile...
319 of 16224 processed. This may take awhile...
320 of 16224 processed. This may take awhile...
321 of 16224 processed. This may take awhile...
322 of 16224 processed. This may take awhile...
323 of 16224 processed. This may take awhile...
324 of 16224 processed. This may take awhile...
325 of 16224 processed. This may take aw

451 of 16224 processed. This may take awhile...
452 of 16224 processed. This may take awhile...
453 of 16224 processed. This may take awhile...
File C:/COUGHVID_public_dataset/public_dataset/0762a05c-1356-4058-af32-ed09fc495f84.webm not found. Trying next extension...
454 of 16224 processed. This may take awhile...
455 of 16224 processed. This may take awhile...
456 of 16224 processed. This may take awhile...
457 of 16224 processed. This may take awhile...
458 of 16224 processed. This may take awhile...
459 of 16224 processed. This may take awhile...
460 of 16224 processed. This may take awhile...
461 of 16224 processed. This may take awhile...
462 of 16224 processed. This may take awhile...
463 of 16224 processed. This may take awhile...
464 of 16224 processed. This may take awhile...
465 of 16224 processed. This may take awhile...
466 of 16224 processed. This may take awhile...
467 of 16224 processed. This may take awhile...
468 of 16224 processed. This may take awhile...
469 of 1622

612 of 16224 processed. This may take awhile...
613 of 16224 processed. This may take awhile...
614 of 16224 processed. This may take awhile...
615 of 16224 processed. This may take awhile...
616 of 16224 processed. This may take awhile...
617 of 16224 processed. This may take awhile...
618 of 16224 processed. This may take awhile...
619 of 16224 processed. This may take awhile...
620 of 16224 processed. This may take awhile...
621 of 16224 processed. This may take awhile...
622 of 16224 processed. This may take awhile...
623 of 16224 processed. This may take awhile...
624 of 16224 processed. This may take awhile...
File C:/COUGHVID_public_dataset/public_dataset/09c0a003-643f-4e0d-a0c6-e1ff8f70ca44.webm not found. Trying next extension...
625 of 16224 processed. This may take awhile...
626 of 16224 processed. This may take awhile...
File C:/COUGHVID_public_dataset/public_dataset/09c410f3-aab8-469d-b4eb-b08f9341fefe.webm not found. Trying next extension...
627 of 16224 processed. This m

...

6328 of 16224 processed. This may take awhile...
6329 of 16224 processed. This may take awhile...
6330 of 16224 processed. This may take awhile...
6331 of 16224 processed. This may take awhile...
6332 of 16224 processed. This may take awhile...
6333 of 16224 processed. This may take awhile...
6334 of 16224 processed. This may take awhile...
6335 of 16224 processed. This may take awhile...
6336 of 16224 processed. This may take awhile...
6337 of 16224 processed. This may take awhile...
6338 of 16224 processed. This may take awhile...
6339 of 16224 processed. This may take awhile...
6340 of 16224 processed. This may take awhile...
6341 of 16224 processed. This may take awhile...
File C:/COUGHVID_public_dataset/public_dataset/63ba6830-f8e2-4b42-8759-56b682fe5bd4.webm not found. Trying next extension...
6342 of 16224 processed. This may take awhile...
6343 of 16224 processed. This may take awhile...
6344 of 16224 processed. This may take awhile...
6345 of 16224 processed. This may take awh

6469 of 16224 processed. This may take awhile...
6470 of 16224 processed. This may take awhile...
6471 of 16224 processed. This may take awhile...
6472 of 16224 processed. This may take awhile...
6473 of 16224 processed. This may take awhile...
6474 of 16224 processed. This may take awhile...
6475 of 16224 processed. This may take awhile...
6476 of 16224 processed. This may take awhile...
File C:/COUGHVID_public_dataset/public_dataset/65b42c62-704a-4800-a9fe-9c07a5e0a0f3.webm not found. Trying next extension...
6477 of 16224 processed. This may take awhile...
File C:/COUGHVID_public_dataset/public_dataset/65be7822-e7c7-4fdc-9549-bf3fac30f919.webm not found. Trying next extension...
6478 of 16224 processed. This may take awhile...
6479 of 16224 processed. This may take awhile...
6480 of 16224 processed. This may take awhile...
File C:/COUGHVID_public_dataset/public_dataset/65c9a70c-232d-41b1-a2a7-279d65d3971d.webm not found. Trying next extension...
6481 of 16224 processed. This may tak

6607 of 16224 processed. This may take awhile...
6608 of 16224 processed. This may take awhile...
6609 of 16224 processed. This may take awhile...
6610 of 16224 processed. This may take awhile...
6611 of 16224 processed. This may take awhile...
6612 of 16224 processed. This may take awhile...
6613 of 16224 processed. This may take awhile...
6614 of 16224 processed. This may take awhile...
6615 of 16224 processed. This may take awhile...
6616 of 16224 processed. This may take awhile...
6617 of 16224 processed. This may take awhile...
6618 of 16224 processed. This may take awhile...
6619 of 16224 processed. This may take awhile...
6620 of 16224 processed. This may take awhile...
6621 of 16224 processed. This may take awhile...
6622 of 16224 processed. This may take awhile...
6623 of 16224 processed. This may take awhile...
6624 of 16224 processed. This may take awhile...
6625 of 16224 processed. This may take awhile...
6626 of 16224 processed. This may take awhile...
6627 of 16224 proces

6764 of 16224 processed. This may take awhile...
6765 of 16224 processed. This may take awhile...
6766 of 16224 processed. This may take awhile...
6767 of 16224 processed. This may take awhile...
6768 of 16224 processed. This may take awhile...
6769 of 16224 processed. This may take awhile...
6770 of 16224 processed. This may take awhile...
6771 of 16224 processed. This may take awhile...
6772 of 16224 processed. This may take awhile...
6773 of 16224 processed. This may take awhile...
6774 of 16224 processed. This may take awhile...
6775 of 16224 processed. This may take awhile...
6776 of 16224 processed. This may take awhile...
6777 of 16224 processed. This may take awhile...
6778 of 16224 processed. This may take awhile...
6779 of 16224 processed. This may take awhile...
6780 of 16224 processed. This may take awhile...
6781 of 16224 processed. This may take awhile...
6782 of 16224 processed. This may take awhile...
6783 of 16224 processed. This may take awhile...
6784 of 16224 proces

6918 of 16224 processed. This may take awhile...
6919 of 16224 processed. This may take awhile...
6920 of 16224 processed. This may take awhile...
6921 of 16224 processed. This may take awhile...
6922 of 16224 processed. This may take awhile...
6923 of 16224 processed. This may take awhile...
6924 of 16224 processed. This may take awhile...
6925 of 16224 processed. This may take awhile...
6926 of 16224 processed. This may take awhile...
6927 of 16224 processed. This may take awhile...
6928 of 16224 processed. This may take awhile...
6929 of 16224 processed. This may take awhile...
6930 of 16224 processed. This may take awhile...
6931 of 16224 processed. This may take awhile...
6932 of 16224 processed. This may take awhile...
6933 of 16224 processed. This may take awhile...
6934 of 16224 processed. This may take awhile...
6935 of 16224 processed. This may take awhile...
6936 of 16224 processed. This may take awhile...
6937 of 16224 processed. This may take awhile...
6938 of 16224 proces

In [63]:
# update dataframe
labelled_data.insert(2,'sounds',sounds)
#labelled_data.insert(3,'sample_rates',sample_rates)

In [ ]:
import pickle as pi

# for me this takes a very long time and generates a 17.2 GB file
#pi.dump(data,open('all_data.pickle','wb'))

# alternatively, use h5py... couldn't get it to work on Windows though
#data.to_hdf('all_data.h5','df',mode='w',format='table',data_columns=True)

#h = h5py.File('all_data.h5')

1

In [ ]:
'''model = tf.keras.applications.resnet50.ResNet50(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None, pooling=None, classes=1000
)'''